In [ ]:
pwd

In [ ]:
import SimpleITK as sitk
import json
from read_roi import read_roi_file, read_roi_zip
from collections import OrderedDict
import glob
import matplotlib
from roifile import ImagejRoi
import os
import copy

In [ ]:
test_p = './test/00673371/'

In [ ]:
####one patient
dcms = sorted(glob.glob(test_p+'/*.dcm'))
print(dcms)
for dr in range(len(dcms)):
    print(dcms[dr])
    oneRois = sorted(glob.glob(dcms[dr][:dcms[dr].rindex('.')]+'_*.roi'))

    print(dcms[dr].split('\\')[-1].split('.')[0]+'_*.roi')
    # print(rois)
    print(len(oneRois))
    for oner in oneRois:

        one_roi = read_roi_file(oner)
        odKey = list(OrderedDict(one_roi).keys())

        if one_roi[odKey[0]]['type']=='rectangle':
            ori_loc= one_roi[odKey[0]]['left'], one_roi[odKey[0]]['top'], one_roi[odKey[0]]['left']+one_roi[odKey[0]]['width'], one_roi[odKey[0]]['top']+ one_roi[odKey[0]]['height']
            
        else:
            ori_loc= min(one_roi[odKey[0]]['x']),  min(one_roi[odKey[0]]['y']),  max(one_roi[odKey[0]]['x']),  max(one_roi[odKey[0]]['y'])

        print(ori_loc)
        

In [ ]:
####one patient
dcmList = sorted(glob.glob(test_p+'/*.dcm'))
for dr in range(len(dcmList)):
    oneRois = sorted(glob.glob(dcms[dr][:dcms[dr].rindex('.')]+'_*.roi'))
    if len(oneRois)!= 0:
        for oner in oneRois:

            one_roi = read_roi_file(oner)
            odKey = list(OrderedDict(one_roi).keys())
            print(odKey)

            if one_roi[odKey[0]]['type']=='rectangle':
                ori_loc= one_roi[odKey[0]]['left'], one_roi[odKey[0]]['top'], one_roi[odKey[0]]['left']+one_roi[odKey[0]]['width'], one_roi[odKey[0]]['top']+ one_roi[odKey[0]]['height']
                
            else:
                ori_loc= min(one_roi[odKey[0]]['x']),  min(one_roi[odKey[0]]['y']),  max(one_roi[odKey[0]]['x']),  max(one_roi[odKey[0]]['y'])

            print(ori_loc)
            with open('temp.json',encoding='UTF8') as json_file:
                json_data = json.load(json_file)
            # for file in roi:
            # print(file)
            # dcm= dcmlist[int(file[1:4])]
            reader = sitk.ReadImage(dcmList[dr])
            SOP_ID = reader.GetMetaData('0008|0018')
            Study_Instance_ID = reader.GetMetaData('0020|000d')
            Series_ID = reader.GetMetaData('0020|000e')
            Study_ID = reader.GetMetaData('0020|0010')
            x_min = ori_loc[0]
            x_max = ori_loc[1]
            y_min = ori_loc[2]
            y_max = ori_loc[3]
            roi_max = "("+str(x_max)+","+str(y_max)+")"
            roi_min = "("+str(x_min)+","+str(y_min)+")"
            json_data['study'][0]['studyId'] = Study_ID
            json_data['study'][0]['studyInstanceUid'] = Study_Instance_ID
            json_data['study'][0]['series'][0]["seriesInstanceUid"] = Series_ID
            templist = json_data['study'][0]['series'][0]['lesion'][0]
            templist["sopInstanceUid"] = SOP_ID
            templist['roi_logicalPixelPos_max'] = roi_max
            templist['roi_logicalPixelPos_min'] = roi_min
            json_data['study'][0]['series'][0]['lesion'].append(templist)
            with open(test_p+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                json.dump(json_data, json_file2, indent="\t")
            with open(test_p+odKey[0]+'.json',encoding='UTF8') as json_file:
                json_data = json.load(json_file)
        del json_data['study'][0]['series'][0]['lesion'][0]
        with open(test_p+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                json.dump(json_data, json_file2, indent="\t")
        print(json_data)
    else:
        pass

In [ ]:
kinds= 'benign'
dataList = './{}/'.format(kinds)
# os.makedirs('./jsonfiles/{}/'.format(kinds))
oneList = sorted(glob.glob(dataList+'*'))
i=0
oneName = oneList[i].split('\\')[-1]
print(oneList)
print(oneName)

In [ ]:
####test sum

kinds= 'malignant'
dataList = './{}/'.format(kinds)
onlyPath = './one_jsonfiles/{}/'.format(kinds)
os.makedirs(onlyPath,exist_ok=True)

wholeList = sorted(glob.glob(dataList+'*'))
print(wholeList)


with open('temp.json',encoding='UTF8') as json_file:
    json_data = json.load(json_file)  
json_temp = []
i = 0


for i in range(len(wholeList)):
    oneName = wholeList[i].split('\\')[-1]
    print('oneName', oneName)
    # savePath = './one_jsonfiles/test_{}/{}/'.format(kinds, oneName)
    # os.makedirs(savePath,exist_ok=True)
    # print('savePath', savePath)
    # test_p = './test/{}/'.format(oneName)
    dcmList = sorted(glob.glob(wholeList[i]+'/*.dcm'))
    print('dcmList:', dcmList)
    for dr in range(len(dcmList)):
        oneRois = sorted(glob.glob(dcmList[dr][:dcmList[dr].rindex('.')]+'_*.roi'))
        if len(oneRois)!= 0:
            for oner in oneRois:
                try:
                    one_roi = read_roi_file(oner)
                    odKey = list(OrderedDict(one_roi).keys())
                    print(odKey)

                    if one_roi[odKey[0]]['type']=='rectangle':
                        ori_loc= one_roi[odKey[0]]['left'], one_roi[odKey[0]]['top'], one_roi[odKey[0]]['left']+one_roi[odKey[0]]['width'], one_roi[odKey[0]]['top']+ one_roi[odKey[0]]['height']
                        
                    else:
                        ori_loc= min(one_roi[odKey[0]]['x']),  min(one_roi[odKey[0]]['y']),  max(one_roi[odKey[0]]['x']),  max(one_roi[odKey[0]]['y'])
                except:
                    print('using ImageJRoi lib')
                    one_roi = ImagejRoi.fromfile(oner)
                    print(one_roi)

                    ori_loc= one_roi.left, one_roi.top, one_roi.right, one_roi.bottom, 

                print(ori_loc)
                # with open('temp.json',encoding='UTF8') as json_file:
                #     json_data = json.load(json_file)
                # for file in roi:
                # print(file)
                # dcm= dcmlist[int(file[1:4])]
                try:
                    reader = sitk.ReadImage(dcmList[dr])
                    SOP_ID = reader.GetMetaData('0008|0018')
                    Study_Instance_ID = reader.GetMetaData('0020|000d')
                    Series_ID = reader.GetMetaData('0020|000e')
                    Study_ID = reader.GetMetaData('0020|0010')
                    x_min = ori_loc[0]
                    x_max = ori_loc[2]
                    y_min = ori_loc[1]
                    y_max = ori_loc[3]
                    roi_max = "("+str(x_max)+","+str(y_max)+")"
                    roi_min = "("+str(x_min)+","+str(y_min)+")"

                    json_data['study'][0]['studyId'] = Study_ID
                    json_data['study'][0]['studyInstanceUid'] = Study_Instance_ID
                    json_data['study'][0]['series'][0]['lesion'][0]["sopInstanceUid"] = SOP_ID
                    json_data['study'][0]['series'][0]["seriesInstanceUid"] = Series_ID
                    json_data['study'][0]['series'][0]['lesion'][0]['roi_logicalPixelPos_max'] = roi_max
                    json_data['study'][0]['series'][0]['lesion'][0]['roi_logicalPixelPos_min'] = roi_min

                    print(json_data['study'][0])
                    jval = copy.deepcopy(json_data['study'][0])
                    json_temp.append(jval)


                except:
                    print('error with open dicom')
                    pass

                print(json_data)

        else:
            pass                    
json_data['study'] = json_temp
with open(onlyPath+'/{}fold_annot.json'.format(kinds),'w',encoding='UTF8') as json_file2:
    json.dump(json_data, json_file2, indent="\t")

In [ ]:
####test each

kinds= 'test'
dataList = './{}/'.format(kinds)
onlyPath = './one_jsonfiles/{}/'.format(kinds)
os.makedirs(onlyPath,exist_ok=True)

wholeList = sorted(glob.glob(dataList+'*'))
print(wholeList)
for i in range(len(wholeList)):
    oneName = wholeList[i].split('\\')[-1]
    print('oneName', oneName)
    savePath = './one_jsonfiles/test_{}/{}/'.format(kinds, oneName)
    os.makedirs(savePath,exist_ok=True)
    print('savePath', savePath)
    # test_p = './test/{}/'.format(oneName)
    dcmList = sorted(glob.glob(wholeList[i]+'/*.dcm'))
    print('dcmList:', dcmList)
    for dr in range(len(dcmList)):
        oneRois = sorted(glob.glob(dcmList[dr][:dcmList[dr].rindex('.')]+'_*.roi'))
        if len(oneRois)!= 0:
            for oner in oneRois:

                one_roi = read_roi_file(oner)
                odKey = list(OrderedDict(one_roi).keys())
                print(odKey)

                if one_roi[odKey[0]]['type']=='rectangle':
                    ori_loc= one_roi[odKey[0]]['left'], one_roi[odKey[0]]['top'], one_roi[odKey[0]]['left']+one_roi[odKey[0]]['width'], one_roi[odKey[0]]['top']+ one_roi[odKey[0]]['height']
                    
                else:
                    ori_loc= min(one_roi[odKey[0]]['x']),  min(one_roi[odKey[0]]['y']),  max(one_roi[odKey[0]]['x']),  max(one_roi[odKey[0]]['y'])

                print(ori_loc)
                with open('temp.json',encoding='UTF8') as json_file:
                    json_data = json.load(json_file)
                # for file in roi:
                # print(file)
                # dcm= dcmlist[int(file[1:4])]
                try:
                    reader = sitk.ReadImage(dcmList[dr])
                    SOP_ID = reader.GetMetaData('0008|0018')
                    Study_Instance_ID = reader.GetMetaData('0020|000d')
                    Series_ID = reader.GetMetaData('0020|000e')
                    Study_ID = reader.GetMetaData('0020|0010')
                    x_min = ori_loc[0]
                    x_max = ori_loc[1]
                    y_min = ori_loc[2]
                    y_max = ori_loc[3]
                    roi_max = "("+str(x_max)+","+str(y_max)+")"
                    roi_min = "("+str(x_min)+","+str(y_min)+")"
                    json_data['study'][0]['studyId'] = Study_ID
                    json_data['study'][0]['studyInstanceUid'] = Study_Instance_ID
                    json_data['study'][0]['series'][0]["seriesInstanceUid"] = Series_ID
                    templist = json_data['study'][0]['series'][0]['lesion'][0]
                    templist["sopInstanceUid"] = SOP_ID
                    templist['roi_logicalPixelPos_max'] = roi_max
                    templist['roi_logicalPixelPos_min'] = roi_min
                    json_data['study'][0]['series'][0]['lesion'].append(templist)
                    with open(savePath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                        json.dump(json_data, json_file2, indent="\t")
                    with open(savePath+odKey[0]+'.json',encoding='UTF8') as json_file:
                        json_data = json.load(json_file)

                    with open(onlyPath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                        json.dump(json_data, json_file2, indent="\t")
                    with open(onlyPath+odKey[0]+'.json',encoding='UTF8') as json_file:
                        json_data = json.load(json_file)
                except:
                    pass
                del json_data['study'][0]['series'][0]['lesion'][0]
                with open(savePath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                        json.dump(json_data, json_file2, indent="\t")
                with open(onlyPath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                        json.dump(json_data, json_file2, indent="\t")
                print(json_data)

        else:
            pass                    


In [ ]:
####all

kinds= 'test'

def making_json(kinds):
    dataList = './{}/'.format(kinds)
    os.makedirs('./jsonfiles/{}/'.format(kinds),exist_ok=True)
    wholeList = sorted(glob.glob(dataList+'*'))

    onlyPath = './jsonfiles/{}/'.format(kinds)
    for i in range(len(wholeList)):
        oneName = wholeList[i].split('\\')[-1]
       
        savePath = './{}/{}/'.format(kinds, oneName)

        dcmList = sorted(glob.glob(wholeList[i]+'/*.dcm'))

        for dr in range(len(dcmList)):
            oneRois = sorted(glob.glob(dcmList[dr][:dcmList[dr].rindex('.')]+'_*.roi'))
            if len(oneRois)!= 0:
                for oner in oneRois:
                    try:
                        one_roi = read_roi_file(oner)
                        odKey = list(OrderedDict(one_roi).keys())

                        if one_roi[odKey[0]]['type']=='rectangle':
                            ori_loc= one_roi[odKey[0]]['left'], one_roi[odKey[0]]['top'], one_roi[odKey[0]]['left']+one_roi[odKey[0]]['width'], one_roi[odKey[0]]['top']+ one_roi[odKey[0]]['height']
                            
                        else:
                            ori_loc= min(one_roi[odKey[0]]['x']),  min(one_roi[odKey[0]]['y']),  max(one_roi[odKey[0]]['x']),  max(one_roi[odKey[0]]['y'])

                        # print(ori_loc)
                        with open('temp.json',encoding='UTF8') as json_file:
                            json_data = json.load(json_file)
                   
                        reader = sitk.ReadImage(dcmList[dr])
                        SOP_ID = reader.GetMetaData('0008|0018')
                        Study_Instance_ID = reader.GetMetaData('0020|000d')
                        Series_ID = reader.GetMetaData('0020|000e')
                        Study_ID = reader.GetMetaData('0020|0010')
                        x_min = ori_loc[0]
                        x_max = ori_loc[1]
                        y_min = ori_loc[2]
                        y_max = ori_loc[3]
                        roi_max = "("+str(x_max)+","+str(y_max)+")"
                        roi_min = "("+str(x_min)+","+str(y_min)+")"
                        json_data['study'][0]['studyId'] = Study_ID
                        json_data['study'][0]['studyInstanceUid'] = Study_Instance_ID
                        json_data['study'][0]['series'][0]["seriesInstanceUid"] = Series_ID
                        templist = json_data['study'][0]['series'][0]['lesion'][0]
                        templist["sopInstanceUid"] = SOP_ID
                        templist['roi_logicalPixelPos_max'] = roi_max
                        templist['roi_logicalPixelPos_min'] = roi_min
                        json_data['study'][0]['series'][0]['lesion'].append(templist)
                        with open(savePath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                            json.dump(json_data, json_file2, indent="\t")
                        with open(savePath+odKey[0]+'.json',encoding='UTF8') as json_file:
                            json_data = json.load(json_file)

                        with open(onlyPath+odKey[0]+'_{}.json'.format(kinds),'w',encoding='UTF8') as json_file2:
                            json.dump(json_data, json_file2, indent="\t")
                        with open(onlyPath+odKey[0]+'_{}.json'.format(kinds),encoding='UTF8') as json_file:
                            json_data = json.load(json_file)
                    except:
                        print('error:',oneName)
                        pass
                    try:
                        del json_data['study'][0]['series'][0]['lesion'][0]
                        with open(savePath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                                json.dump(json_data, json_file2, indent="\t")
                        with open(onlyPath+odKey[0]+'_{}.json'.format(kinds),'w',encoding='UTF8') as json_file2:
                                json.dump(json_data, json_file2, indent="\t")
                        # print(json_data)
                    except:
                        pass


            else:
                pass                    


In [ ]:
benign = making_json(kinds='benign')
malig = making_json(kinds='malignant')
overlap = making_json(kinds='overlap')

In [ ]:
def making_json(kinds):
    dataList = './{}/'.format(kinds)
    os.makedirs('./jsonfiles/{}/'.format(kinds),exist_ok=True)
    wholeList = sorted(glob.glob(dataList+'*'))

    onlyPath = './jsonfiles/{}/'.format(kinds)
    for i in range(len(wholeList)):
        oneName = wholeList[i].split('\\')[-1]
       
        savePath = './{}/{}/'.format(kinds, oneName)

        dcmList = sorted(glob.glob(wholeList[i]+'/*.dcm'))

        for dr in range(len(dcmList)):
            oneRois = sorted(glob.glob(dcmList[dr][:dcmList[dr].rindex('.')]+'_*.roi'))
            if len(oneRois)!= 0:
                for oner in oneRois:
                    try:
                        one_roi = read_roi_file(oner)
                        odKey = list(OrderedDict(one_roi).keys())

                        if one_roi[odKey[0]]['type']=='rectangle':
                            ori_loc= one_roi[odKey[0]]['left'], one_roi[odKey[0]]['top'], one_roi[odKey[0]]['left']+one_roi[odKey[0]]['width'], one_roi[odKey[0]]['top']+ one_roi[odKey[0]]['height']
                            
                        else:
                            ori_loc= min(one_roi[odKey[0]]['x']),  min(one_roi[odKey[0]]['y']),  max(one_roi[odKey[0]]['x']),  max(one_roi[odKey[0]]['y'])

                        # print(ori_loc)
                        with open('temp.json',encoding='UTF8') as json_file:
                            json_data = json.load(json_file)
                   
                        reader = sitk.ReadImage(dcmList[dr])
                        SOP_ID = reader.GetMetaData('0008|0018')
                        Study_Instance_ID = reader.GetMetaData('0020|000d')
                        Series_ID = reader.GetMetaData('0020|000e')
                        Study_ID = reader.GetMetaData('0020|0010')
                        x_min = ori_loc[0]
                        x_max = ori_loc[1]
                        y_min = ori_loc[2]
                        y_max = ori_loc[3]
                        roi_max = "("+str(x_max)+","+str(y_max)+")"
                        roi_min = "("+str(x_min)+","+str(y_min)+")"
                        json_data['study'][0]['studyId'] = Study_ID
                        json_data['study'][0]['studyInstanceUid'] = Study_Instance_ID
                        json_data['study'][0]['series'][0]["seriesInstanceUid"] = Series_ID
                        templist = json_data['study'][0]['series'][0]['lesion'][0]
                        templist["sopInstanceUid"] = SOP_ID
                        templist['roi_logicalPixelPos_max'] = roi_max
                        templist['roi_logicalPixelPos_min'] = roi_min
                        json_data['study'][0]['series'][0]['lesion'].append(templist)
                        # with open(savePath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                        #     json.dump(json_data, json_file2, indent="\t")
                        # with open(savePath+odKey[0]+'.json',encoding='UTF8') as json_file:
                        #     json_data = json.load(json_file)

                        with open(onlyPath+'{}_{}.json'.format(oneName, odKey[0]),'w',encoding='UTF8') as json_file2:
                            json.dump(json_data, json_file2, indent="\t")
                        with open(onlyPath+'{}_{}.json'.format(oneName, odKey[0]),encoding='UTF8') as json_file:
                            json_data = json.load(json_file)
                    except:
                        print('error:',oneName)
                        pass
                    try:
                        del json_data['study'][0]['series'][0]['lesion'][0]
                        # with open(savePath+odKey[0]+'.json','w',encoding='UTF8') as json_file2:
                        #         json.dump(json_data, json_file2, indent="\t")
                        with open(onlyPath+'{}_{}.json'.format(oneName, odKey[0]),'w',encoding='UTF8') as json_file2:
                                json.dump(json_data, json_file2, indent="\t")
                        # print(json_data)
                    except:
                        pass


            else:
                pass                    


In [ ]:
benign = making_json(kinds='benign')
malig = making_json(kinds='malignant')
overlap = making_json(kinds='overlap')

In [ ]:
33333

In [ ]:
json_data['study'][0]['series'][0]['lesion'][4]

In [ ]:
roiname = 'CT/09902573/abnormal.zip'
os.path.dirname(test_p)
dcmlist = glob.glob(test_p+'/*.dcm')

In [ ]:
print(os.path.dirname(test_p))

In [ ]:
for roiname in glob.iglob('**/*.roi',recursive=True):
    roipath = os.path.dirname(roiname)
    dcmlist = glob.glob(roipath+'/*.dcm')
    with open('temp.json',encoding='UTF8') as json_file:
        json_data = json.load(json_file)
    for file in roi:
        print(file)
        dcm= dcmlist[int(file[1:4])]
        reader = sitk.ReadImage(dcm)
        SOP_ID = reader.GetMetaData('0008|0018')
        Study_Instance_ID = reader.GetMetaData('0020|000d')
        Series_ID = reader.GetMetaData('0020|000e')
        Study_ID = reader.GetMetaData('0020|0010')
        x_min = roi[file]['left']
        x_max = roi[file]['left']+roi[file]['width']
        y_min = roi[file]['top']
        y_max = roi[file]['top']+roi[file]['height']
        roi_max = "("+str(x_max)+","+str(y_max)+")"
        roi_min = "("+str(x_min)+","+str(y_min)+")"
        json_data['study'][0]['studyId'] = Study_ID
        json_data['study'][0]['studyInstanceUid'] = Study_Instance_ID
        json_data['study'][0]['series'][0]["seriesInstanceUid"] = Series_ID
        templist = json_data['study'][0]['series'][0]['lesion'][0]
        templist["sopInstanceUid"] = SOP_ID
        templist['roi_logicalPixelPos_max'] = roi_max
        templist['roi_logicalPixelPos_min'] = roi_min
        json_data['study'][0]['series'][0]['lesion'].append(templist)
        with open(roiname[:-4]+'.json','w',encoding='UTF8') as json_file2:
            json.dump(json_data, json_file2, indent="\t")
        with open(roiname[:-4]+'.json',encoding='UTF8') as json_file:
            json_data = json.load(json_file)
    del json_data['study'][0]['series'][0]['lesion'][0]
    with open(roiname[:-4]+'.json','w',encoding='UTF8') as json_file2:
            json.dump(json_data, json_file2, indent="\t")
    print(json_data)
    

In [ ]:
roi = 

In [ ]:
roi.bottom

In [ ]:
with open('temp.json',encoding='UTF8') as json_file:
    json_data = json.load(json_file)
    print(type(json_data['study'][0]['series'][0]['lesion']))
    print(type(templist))